<a href="https://colab.research.google.com/github/dheerajreddy2020/Deep-Neural-Networks-DNN-/blob/master/Creating_Deep_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understanding the building blocks of Deep Neural Networks
* This notebook helps you in understanding how to develop a deep neural network.
* What goes on behind the Dense commands in Keras/Tensorflow.
* How to optimize the parameters using gradient descent method.

In [0]:
import numpy as np
import matplotlib.pyplot as plt

## Create activation functions and their gradients

In [0]:
def sigmoid(z):
  return (1/(1+np.exp(-z)))

def relu(Z):
  Z[Z<0]=0
  return Z

def sigmoid_backward(Z):
  return sigmoid(Z)*(1-sigmoid(Z))

def relu_backward(Z):
  Z[Z<=0]=0
  Z[Z>0]=1
  return Z

## 1. Initializing the parameters

In [0]:
#This function works similar to kernel initializer = 'glorot_normal' and bias_initilizer= 'zeros' in keras
def initialize_parameters(dims):
  '''
  input an array of dimensions in each layer 
  Eg dims= [10,8,5,2] denotes 10 input parameters, 
  8 neurons in 1st hidden layer, 
  5 neurons in 2nd hidden layer,
  2 neurons in output layer.
  Returns a dictionary of parameters W(Kernel) & b(bias)
  '''
  params={}
  for i in range(1,len(dims)):
    params["W"+str(i)]=np.random.randn(dims[i],dims[i-1])/np.sqrt(dims[i-1])
    params["b"+str(i)]=np.zeros((dims[i],1))
  return params

##2. Create forward Propagation function

In [0]:
def model_forward(X,params):
  '''
  X is the input parameters
  params is a dictionary of parameters 
  Returns dictionaries of outputs and activations for all the layers Z &
  '''
  net_len=int(len(params)/2)
  Z={}
  A={}
  A[str(0)]=X
  for i in range(1,net_len):
    Z[str(i)]=np.dot(params["W"+str(i)],A[str(i-1)])+params["b"+str(i)]
    A[str(i)]=relu(Z[str(i)])
  
  Z[str(net_len)]=np.dot(params["W"+str(net_len)],A[str(net_len-1)])+params["b"+str(net_len)]
  A[str(net_len)]=sigmoid(Z[str(net_len)])
  return A,Z

## 3. Calculating Cost/ loss function

In [0]:
def cost(Y_pred,Y_act,n_outputs):
  '''
  Make sure the first argument is the predicted Y and 2nd argument is actual Y
  Returns Cost or loss of the current prediction
  '''
  m=Y_pred.shape[1]
  if n_outputs>1:
    cost=-1/m*np.sum(Y_act*np.log(Y_pred))
  else:
    cost=-1/m*np.sum(Y_act*np.log(Y_pred)+(1-Y_act)*np.log(1-Y_pred))
  return cost

## 4. Create Backward propagation functions
* This is a very important step which helps in updating the parameters.

In [0]:
def model_backward(Y,A,Z,params):
  '''
  Y- Actual outputs in the last layer
  Give dictionaries of 
  A- Dictionary with activations of each layer
  Z- Estimated values before activation
  params- Parameters dictionary consisting of W and b
  Returns gradients (grads) and cache(consists of da and dz)
  '''
  dz={}
  da={}
  grads={}
  l=len(Z)
  m=Y.shape[1]
  dz[str(l)]=A[str(l)]-Y
  for i in reversed(range(1,l+1)):
    grads['dW'+str(i)]=1/m*np.dot(dz[str(i)],A[str(i-1)].T)
    grads['db'+str(i)]=1/m*np.sum(dz[str(i)],axis=1,keepdims=True)
    if i>1:
      da[str(i-1)]=np.dot(params['W'+str(i)].T,dz[str(i)])
      dz[str(i-1)]=da[str(i-1)]*relu_backward(Z[str(i-1)])
  cache=da,dz
  return grads,cache

## 5. Create function to update all the parameters
* Here a Gradient design optimization method is used to optimize the parameters for each step
* However there are many advanced optimization methods.
* RMS Prop and Adam optimizer are frequently used in general

In [0]:
def update_params(params,grads,alpha,n_layers):
  '''
  params- Parameters dictionary
  grads- grads dictionary
  alpha- learning rate
  n_layers- number of layers of deep learning network
  Returns dictionary of updated paramters
  '''
  for i in range(1,n_layers+1):
    params["W"+str(i)]=params["W"+str(i)]-alpha*grads['dW'+str(i)]
    params["b"+str(i)]=params["b"+str(i)]-alpha*grads['db'+str(i)]
  return params


## 6. Building a n-layered model

In [0]:
def create_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):#lr was 0.009
    """
    Implements a n-layer neural network: [LINEAR->RELU]*(n-1)->LINEAR->SIGMOID.
    
    Arguments:
    X -- data, numpy array of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    layers_dims -- list containing the input size and each layer size, of length (number of layers + 1).
    learning_rate -- learning rate of the gradient descent update rule
    num_iterations -- number of iterations of the optimization loop
    print_cost -- if True, it prints the cost every 100 steps
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """

    costs = []                         # keep track of cost
    
    # Parameters initialization. (≈ 1 line of code)
    params = initialize_parameters(layers_dims)
    n_layers=len(layers_dims)-1
    alpha=learning_rate
    for i in range(0, num_iterations):

        # Forward propagation: [LINEAR -> RELU]*(L-1) -> LINEAR -> SIGMOID.
        A, Z = model_forward(X,params)
        
        # Compute cost.
        cost = cal_cost(A[str(n_layers)],Y,1)
    
        # Backward propagation.
        grads,cache = model_backward(Y,A,Z,params)
 
        # Update parameters.
        params = update_params(params,grads,alpha,n_layers)
                
        # Print the cost every 100 training example
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
            costs.append(cost)
        
            
    # plot the cost
    plt.plot(np.squeeze(costs))
    plt.ylabel('cost')
    plt.xlabel('iterations (per hundreds)')
    plt.title("Learning rate =" + str(learning_rate))
    plt.show()
    
    return params